<a href="https://colab.research.google.com/github/Satwikram/Deep-Learning-Notebooks/blob/master/CNN/Face_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
"""
@Author: Satwik Ram K

Face Emotion Detection using Tensorflow

"""
# Importing Libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [23]:
num_class = 6
batch_size = 64
img_size = (48, 48, 1)

In [ ]:
! unzip /content/fer2013_1.zip

In [24]:
train_data_dir = '/content/fer2013_1/train'
val_data_dir = '/content/fer2013_1/validation'

In [25]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, Activation, Flatten

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [27]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    fill_mode = 'nearest',
    rescale = 1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
)

In [28]:
val_datagen = ImageDataGenerator(rescale=1./255)


In [29]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode = 'grayscale',
    target_size = img_size[:2],
    batch_size = batch_size,
    class_mode = 'categorical' 
)

Found 28273 images belonging to 6 classes.


In [30]:
val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    color_mode = 'grayscale',
    target_size = img_size[:2],
    batch_size = batch_size,
    class_mode = 'categorical' 
)

Found 3534 images belonging to 6 classes.


In [31]:
from tensorflow.keras.layers import ELU


Building Model

In [32]:
def build_model():

  model = Sequential()

  model.add(Conv2D(32, (5, 5), padding = "same", input_shape = img_size))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  # Block #2: second CONV => RELU => CONV => RELU => POOL
  # layer set
  model.add(Conv2D(64, (5, 5), padding="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(256, (5, 5), padding="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  # Block #3: third CONV => RELU => CONV => RELU => POOL
  # layer set
  model.add(Conv2D(256, (5, 5), padding="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(512, (5, 5), padding="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  # Block #4: third CONV => RELU => CONV => RELU => POOL
  # layer set
  model.add(Conv2D(1024, (5, 5), padding="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(1024, (5, 5), padding="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  # Block #5: first set of FC => RELU layers
  model.add(Flatten())
  model.add(Dense(1024))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  # Block #6: second set of FC => RELU layers
  model.add(Dense(2048))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  # Block #7: softmax classifier
  model.add(Dense(num_class))
  model.add(Activation("softmax"))

  return model

In [33]:
model = build_model()

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 48, 48, 32)        832       
_________________________________________________________________
activation_10 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 64)        51264     
_________________________________________________________________
activation_11 (Activation)   (None, 24, 24, 64)       

In [35]:
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

                     
checkpoint = ModelCheckpoint("emotions.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)



In [36]:
earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)


In [37]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]


In [38]:
# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics = ['accuracy'])


In [39]:
nb_train_samples = 28291
nb_validation_samples = 3534
epochs = 10

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = val_generator,
    validation_steps = nb_validation_samples // batch_size)

In [ ]:
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

nb_train_samples = 28273
nb_validation_samples = 3534

# We need to recreate our validation generator with shuffle = false
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        color_mode = 'grayscale',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())

#Confution Matrix and Classification Report
Y_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
cnf_matrix = confusion_matrix(validation_generator.classes, y_pred)

plt.imshow(cnf_matrix, interpolation='nearest')
plt.colorbar()
tick_marks = np.arange(len(classes))
_ = plt.xticks(tick_marks, classes, rotation=90)
_ = plt.yticks(tick_marks, classes)